In [11]:
!python -m zipfile -e kalp-atis.zip . 

In [13]:
!ls Heartbeat_Sound/

artifact  extrahls  extrastole	murmur	normal	unlabel




1. **Artifact (Artefakt)**: Tıbbi görüntüleme veya ses kaydı gibi teknolojik süreçlerde, istenmeyen veya yanlış sonuçlara yol açabilen dış etkenler veya hatalar anlamına gelir. Örneğin, elektrokardiyogram (EKG) çekimlerinde hareket artefaktları veya elektrotların yerleştirilmesinden kaynaklanan gürültüler olabilir.

2. **Extrahls (Ekstra Sistol)**: Kalp atışlarının normal döngüsünden farklı olarak meydana gelen erken bir kalp kasılmadır. Normal kalp atışı sırasında ani bir ekstra kasılma ile karakterizedir. Bu durum bazen kalp atışı hissi, göğüs ağrısı veya baş dönmesi gibi semptomlara yol açabilir.

3. **Extrastole (Ekstra Sistol)**: Bu terim de "Extrahls" ile aynı anlama gelir. Kalbin normal atım döngüsünden önce gelen ek bir kasılma olarak tanımlanır.

4. **Murmur (Üfleme)**: Kalp seslerinden biri olan ve genellikle anormal kan akışı nedeniyle oluşan bir ses. Üfleme, genellikle stetoskopla dinlendiğinde duyulabilir ve kalbin normal seslerinden farklı bir ses olarak tanımlanır.



In [47]:
#Veri setini temizle

import os
import shutil
from glob import glob

files = ["extrastole","murmur","normal","unlabel"]

dataset_files = glob("Heartbeat_Sound/*")

print(f"dataset_files = {dataset_files}")

for file in dataset_files:
    filename = file.split(os.sep)[-1]
    
    if not filename in files:
        print(f"Dosya Siliniyor.. {file}")
        shutil.rmtree(file,ignore_errors=True)



dataset_files = ['Heartbeat_Sound/extrastole', 'Heartbeat_Sound/unlabel', 'Heartbeat_Sound/normal', 'Heartbeat_Sound/murmur']


In [29]:
dataset_files = glob("Heartbeat_Sound/*")
for file in dataset_files:
    print(f"{file} : {len(os.listdir(file))}")

Heartbeat_Sound/extrastole : 46
Heartbeat_Sound/unlabel : 247
Heartbeat_Sound/normal : 352
Heartbeat_Sound/murmur : 129


In [74]:
import numpy as np
import librosa 


def load_files(files,duration = 10, sr = 22050):
    mfccs = 0
    input_length = sr*duration
    data = []
    for sound_file in files:
        
        X, sr = librosa.load(sound_file,sr = sr, duration = duration)
        dur = librosa.get_duration(y = X, sr = sr)
        if round(dur) < duration:
            y = librosa.util.fix_length(X,size = input_length)
        mfccs = np.mean(librosa.feature.mfcc(y = X, sr = sr, n_mfcc = 25).T,axis = 0)
        
        feature = np.array(mfccs).reshape([-1,1])
        data.append(feature)

    return data


In [75]:
dataset_files

['Heartbeat_Sound/extrastole',
 'Heartbeat_Sound/unlabel',
 'Heartbeat_Sound/normal',
 'Heartbeat_Sound/murmur']

In [88]:
extrastole_files = glob(os.sep.join([dataset_files[0],'*.wav']))
extrastole_sounds = load_files(extrastole_files)
extrastole_labels = [2] * len(extrastole_sounds)

In [94]:
unlabel_files = glob(os.sep.join([dataset_files[1],'*.wav']))
unlabel_sounds = load_files(unlabel_files)
unlabel_labels = [-1] * len(unlabel_sounds)

In [95]:
normal_files = glob(os.sep.join([dataset_files[2],'*.wav']))
normal_sounds = load_files(normal_files)
normal_labels = [2] * len(normal_sounds)

In [93]:
murmur_files = glob(os.sep.join([dataset_files[3],'*.wav']))
murmur_sounds = load_files(murmur_files)
murmur_labels = [1] * len(murmur_sounds)

In [7]:
x_data = np.concatenate((normal_sounds,murmur_files,extrastole_sounds))
y_data = np.concatenate((normal_labels,murmur_labels,extrastole_sounds))

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (46,) + inhomogeneous part.

In [15]:
len(normal_sounds),len(murmur_files),len(extrastole_sounds)

(351, 129, 46)

In [16]:
# -*- coding: utf-8 -*-
"""
Created on Sat Jun 26 22:36:11 2024

@author: UK
"""
import numpy as np
import os
import fnmatch
import librosa

data_path = "./Heartbeat_Sound/"
tarin_data = data_path
unlabel_data = data_path + "/unlabel/"
normal_data = tarin_data + '/normal/'
murmur_data = tarin_data + '/murmur/'
extrastole_data = tarin_data + '/extrastole/'

print("Normal kalp sesi dosyalari:", len(os.listdir(normal_data)))
print("Murmur kalp sesi dosyalari:",len(os.listdir(murmur_data)))
print("Extrastole kalp sesi dosyalari", len(os.listdir(extrastole_data)))
print('Egitim sureci icin toplam ses dosyasi:', len(os.listdir(normal_data))
                              + len(os.listdir(murmur_data))
                              + len(os.listdir(extrastole_data)))
print("Test sureci icin toplam ses dosyasi: ", len(os.listdir(unlabel_data)))

def load_file_data(folder, file_names, duration=10, sr=22050):
    mfccs = 0
    input_length=sr*duration
    data = []
    for file_name in file_names:
        try:
            sound_file=folder+file_name
            print ("Okunan dosya:",sound_file)
            X, sr = librosa.load( sound_file, sr=sr, duration=duration)
            dur = librosa.get_duration(y=X, sr=sr)
            if (round(dur) < duration):
                print ("uzunluk duzeltme:",file_name)
                y = librosa.util.fix_length(X, input_length)
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sr, n_mfcc=25).T,axis=0)
        except Exception as e:
            print("Dosya okuma hatasi!:",file_name)
        feature = np.array(mfccs).reshape([-1,1])
        data.append(feature)
    return data

normal_files = fnmatch.filter(os.listdir(normal_data), 'normal*.wav')
normal_sounds = load_file_data(folder=normal_data,file_names=normal_files, duration=10)
normal_labels = [2 for items in normal_sounds]
murmur_files = fnmatch.filter(os.listdir(murmur_data), 'murmur*.wav')
murmur_sounds = load_file_data(folder=murmur_data,file_names=murmur_files, duration=10)
murmur_labels = [1 for items in murmur_files]
extrastole_files = fnmatch.filter(os.listdir(extrastole_data), 'extrastole*.wav')
extrastole_sounds = load_file_data(folder=extrastole_data,file_names=extrastole_files, duration=10)
extrastole_labels = [2 for items in extrastole_files]

Bunlabelledtest_files = fnmatch.filter(os.listdir(unlabel_data), 'Bunlabelledtest*.wav')
Bunlabelledtest_sounds = load_file_data(folder=unlabel_data,file_names=Bunlabelledtest_files, duration=10)
Bunlabelledtest_labels = [-1 for items in Bunlabelledtest_sounds]

x_data = np.concatenate((normal_sounds,murmur_sounds,extrastole_sounds))
y_data = np.concatenate((normal_labels,murmur_labels,extrastole_labels))

test_x = Bunlabelledtest_sounds
test_y = Bunlabelledtest_labels

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Bidirectional, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, train_size=0.8, random_state=42, shuffle=True)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.8, random_state=42, shuffle=True)

y_train = np.array(tf.keras.utils.to_categorical(y_train, 3))
y_test = np.array(tf.keras.utils.to_categorical(y_test, 3))
y_val = np.array(tf.keras.utils.to_categorical(y_val, 3))
test_y=np.array(tf.keras.utils.to_categorical(test_y, 3))

#LSTM Tanimi
model = Sequential()
model.add(Bidirectional(LSTM(128, dropout=0.05, recurrent_dropout=0.20, 
                             return_sequences=True), input_shape = (25,1)))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Flatten())
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=Adam(1e-4), metrics=['acc'])
model.summary()

history=model.fit(x_train, y_train,
                  batch_size=3,
                  epochs=20,
                  validation_data=(x_val, y_val))

y_pred = model.predict(x_test, batch_size=5)
scores = model.evaluate(x_test, y_test)
print ("Model Dogruluk (Accuracy): ", round(scores[1]*100),"%")

def preprocess_audio(file_path, duration=10, sr=22050):
    input_length = sr * duration
    process_file = []
    X, sr = librosa.load(file_path, sr=sr, duration=duration)
    dur = librosa.get_duration(y=X, sr=sr)
    if round(dur) < duration:
        y = librosa.util.fix_length(X, input_length)
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sr, n_mfcc=25).T, axis=0)
    feature = np.array(mfccs).reshape([-1, 1])
    process_file.append(feature)
    process_file_array = np.asarray(process_file)
    return process_file_array

audio_file_path = 'data/normal/normal__103_1305031931979_D1.wav'
processed_audio = preprocess_audio(audio_file_path)

prediction = model.predict(processed_audio)

predicted_class = np.argmax(prediction)

class_mapping = {0: 'abnormal', 1: 'abnormal', 2: 'normal'}
predicted_label = class_mapping[predicted_class]

# Print the result
print('Ses dosyasi '+audio_file_path+' icin tahmin edilen class / sinif: '+predicted_label)





Normal kalp sesi dosyalari: 352
Murmur kalp sesi dosyalari: 129
Extrastole kalp sesi dosyalari 46
Egitim sureci icin toplam ses dosyasi: 527
Test sureci icin toplam ses dosyasi:  247
Okunan dosya: ./Heartbeat_Sound//normal/normal_noisynormal_136_1306429977501_D1.wav
Okunan dosya: ./Heartbeat_Sound//normal/normal__178_1307989887769_D.wav
Okunan dosya: ./Heartbeat_Sound//normal/normal_noisynormal_109_1305653646620_A.wav
Okunan dosya: ./Heartbeat_Sound//normal/normal__146_1306778707532_D1.wav
Okunan dosya: ./Heartbeat_Sound//normal/normal__126_1306777102824_D.wav
uzunluk duzeltme: normal__126_1306777102824_D.wav
Dosya okuma hatasi!: normal__126_1306777102824_D.wav
Okunan dosya: ./Heartbeat_Sound//normal/normal__154_1306935608852_A1.wav
uzunluk duzeltme: normal__154_1306935608852_A1.wav
Dosya okuma hatasi!: normal__154_1306935608852_A1.wav
Okunan dosya: ./Heartbeat_Sound//normal/normal__129_1306344506305_D1.wav
uzunluk duzeltme: normal__129_1306344506305_D1.wav
Dosya okuma hatasi!: normal_

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (129,) + inhomogeneous part.

In [21]:
np.concatenate((np.random.randn(3,1),np.random.randn(4,1)))

array([[ 1.10912682],
       [ 2.08081011],
       [ 0.2549933 ],
       [-0.48258354],
       [ 1.44389794],
       [-0.53642025],
       [-0.60050373]])

In [26]:
import numpy as np
import librosa
import os

def extract_features(file_path):
    audio, sample_rate = librosa.load(file_path, res_type='kaiser_fast')
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled = np.mean(mfccs.T, axis=0)
    return mfccs_scaled

# Örnek veri seti
data_path = "./Heartbeat_Sound/normal/"
labels = []
features = []

for file_name in os.listdir(data_path):
    if file_name.endswith(".wav"):
        class_label = file_name.split("_")[0]  # Örnek: "dog_bark_1.wav" dosyasından "dog" sınıfı
        file_path = os.path.join(data_path, file_name)
        mfccs = extract_features(file_path)
        features.append(mfccs)
        labels.append(class_label)

# Verileri numpy dizilerine dönüştürme
X = np.array(features)
y = np.array(labels)

# Label Encoding
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

le = LabelEncoder()
y = le.fit_transform(y)
y = to_categorical(y)


ModuleNotFoundError: No module named 'resampy'

This error is lazily reported, having originally occured in
  File /root/venv/lib/python3.11/site-packages/librosa/core/audio.py, line 33, in <module>

----> resampy = lazy.load("resampy")

In [23]:
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 2.0 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 9.1 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 9.9 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.7/80.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 11.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 11.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.